# Notebook to test 

## Jupyter tools

In [4]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False

## Import Libraries

In [5]:
import rasterio
import matplotlib.pyplot as plt

## Get Paths to Tiffs

In [6]:
import os
import numpy as np
import xarray as xr

def return_path_dataarray():
    incomplete_years = [1984, 1994, 2002, 2003, 2008]
    years = [year for year in range(1984, 2021) if year not in incomplete_years]
    im_type = ["hab", "chern"]
    month_groups = ["JFM", "AMJ", "JAS", "OND"]
    directory = (
        "/gws/nopw/j04/ai4er/guided-team-challenge/2021/biodiversity/gee_satellite_data"
    )
    # year, month_group, im_type
    path_array = np.empty([len(years), len(month_groups), len(im_type)], dtype=object)
    for year in years:
        if year not in incomplete_years:
            path = os.path.join(directory, str(year))
            for i in os.listdir(path):
                full_name = os.path.join(path, i)
                coord_list = [years, month_groups, im_type]
                indices = []
                for coord_no, coord in enumerate(coord_list):
                    for counter, value in enumerate(coord):
                        if str(value) in full_name:
                            indices.append(counter)
                path_array[indices[0], indices[1], indices[2]] = full_name
    da = xr.DataArray(
        data=path_array,
        dims=["yr", "mn", "ty"],
        coords=dict(
            yr=years,
            mn=month_groups,
            ty=im_type,
        ),
        attrs=dict(
            description="Paths to tif.",
        ),
    )
    return da

path_da = return_path_dataarray()

## First attempt at getting x values.

In [ ]:
import numpy as np

def return_x_values(path_da, year):
    """This seems to use up too much RAM"""
    x_list = []
    for mn in path_da.coords['mn'].values:
        for ty in path_da.coords['ty'].values:
            image = rasterio.open(path_da.sel(yr=year, mn=mn, ty=ty).values.tolist())
            x_list.append(image.read().ravel())
    return np.asarray(x_list)

return_x_values(path_da, 2000)